In [ ]:
from tw_stock import tw_stock
import pandas as pd
import datetime
import numpy as np
import matplotlib.pyplot as plt

# === 初始化資料 ===
tw = tw_stock()
start = datetime.datetime(2023, 12, 1)
end = datetime.datetime(2024, 3, 1)

close = tw.get("close", start, end)
high = tw.get("high", start, end)
low = tw.get("low", start, end)
volume = tw.get("volume", start, end)

# 均線資料
def MA(df, n): return df.rolling(n).mean()

bbi短 = (MA(close, 8) + MA(close, 13) + MA(close, 21) + MA(close, 34)) / 4
bbi長 = (MA(close, 55) + MA(close, 89) + MA(close, 144) + MA(close, 233)) / 4
量5 = MA(volume, 5)
量20 = MA(volume, 20)

# === 策略函數：找進場 ===
def bbi策略進場(close, high, low, bbi短, bbi長, 量5, 量20, x):
    c1 = low.iloc[x] > bbi長.iloc[x]
    c2 = high.iloc[x-1] < bbi短.iloc[x-1]
    c3 = high.iloc[x] > bbi短.iloc[x]
    c4 = 量5.iloc[x] > 量20.iloc[x]
    c5 = 量5.iloc[x] > 量5.iloc[x-1]
    match = c1 & c2 & c3 & c4 & c5
    return close.iloc[x][match].index.values

# === 回測參數 ===
觀察天數 = 5
result = []

for i in range(250, len(close) - 觀察天數):
    buy_date = close.index[i]
    try:
        stocks = bbi策略進場(close, high, low, bbi短, bbi長, 量5, 量20, i)
        for stock in stocks:
            buy_price = close.iloc[i][stock]
            future_lows = low.iloc[i+1:i+1+觀察天數][stock].values
            future_closes = close.iloc[i+1:i+1+觀察天數][stock].values
            future_bbi短 = bbi短.iloc[i+1:i+1+觀察天數][stock].values
            if np.isnan(buy_price) or np.isnan(future_bbi短).any():
                continue

            出場日 = None
            for j in range(len(future_lows)):
                if future_lows[j] < future_bbi短[j]:
                    出場日 = close.index[i+1+j]
                    exit_price = future_closes[j]
                    break
            else:
                出場日 = close.index[i+觀察天數]
                exit_price = future_closes[-1]

            pct = (exit_price - buy_price) / buy_price
            result.append({
                "股票": stock,
                "進場日": buy_date,
                "出場日": 出場日,
                "進場價": round(buy_price, 2),
                "出場價": round(exit_price, 2),
                "報酬率": round(pct * 100, 2),
                "持有天數": (出場日 - buy_date).days
            })
    except Exception as e:
        print(f"{buy_date} 當天策略錯誤：{e}")
        continue

# === 統計與顯示 ===
df_result = pd.DataFrame(result)
df_result["結果"] = df_result["報酬率"].apply(lambda x: "勝利" if x > 0 else "失敗")

import seaborn as sns
sns.set_theme()
plt.rcParams['font.family'] = 'Microsoft JhengHei'  # 中文字體設定（Windows）
plt.rcParams['axes.unicode_minus'] = False

plt.figure(figsize=(8, 5))
sns.countplot(data=df_result, x='結果')
plt.title("BBI 策略勝敗分布")
plt.ylabel("次數")
plt.grid(True, linestyle='dotted')
plt.show()

